# Model comparison

In this notebook we will compare PEC corrections calculated with different ol-melt Fe-Mg Kd models

import the relevant packages

In [13]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

from IPython.display import clear_output

load data and set up FeOi according to the [FeOi](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) and [PEC correction](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html) examples.

In [14]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"
pressure_file ="./data/pressure.csv"
wholerock_file = "./data/wholerock.csv"

melt = mp.read_melt(melt_file, index_col=["name"], units="wt. %")
olivine = mp.read_olivine(olivine_file, index_col=["name"], units="wt. %")
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

In [15]:
x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)
FeO_model = FeOi_predict.model

Here are all the Kd models we're going to test. You can also do this for Fe3Fe2 or melt thermometers instead with `mpc.Fe3Fe2_models` or `mpc.melt_thermometers`. 

In [16]:
Kd_models = mpc.Kd_ol_FeMg_models
Kd_models

['blundy2020',
 'putirka2016_8a',
 'putirka2016_8b',
 'putirka2016_8c',
 'putirka2016_8d',
 'saper2022',
 'sun2020',
 'toplis2005']

Now we calculate PEC corrections iteratively for each Kd model. Crystallization extents are stored in the `pec_results` dataframe and corrected melt compositions in the `corrected_inclusions` dictionary. This dictionary uses Kd model names as keys and stores compositions in dataframes.

In [17]:
pec_results = pd.DataFrame(index=melt.index, dtype=float)
corrected_inclusions = {}

for i, model in enumerate(Kd_models):

    mpc.model_configuration.Kd_model = model
    clear_output()
    print(f"model: {model}\n{i+1:03}/{len(Kd_models):03}")

    pec_model = mpc.PEC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeO_model)
    melts_corrected, pec, checks = pec_model.correct()

    pec_results[model] = pec["total_crystallisation"]
    corrected_inclusions[model] = melts_corrected.copy()
    

model: toplis2005
008/008
Equilibrating ... |██████████████████████████████| 100% [10/10] in 4.1s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 14.3s 


These are the crystallization extents calculated with the different models

In [18]:
results

,blundy2020,putirka2016_8a,putirka2016_8b,putirka2016_8c,putirka2016_8d,saper2022,sun2020,toplis2005
name,,,,,,,,
PI032-04-01,9.615161,10.931458,10.835327,10.940723,11.450806,6.979077,11.060718,10.001428
PI032-04-02,11.017993,12.268896,12.265845,12.265845,12.456689,8.184579,12.224097,11.355652
PI041-02-02,1.159460,2.129163,2.130688,2.327637,1.930688,-0.800439,1.762512,0.383655
PI041-03-01,14.807861,16.084595,15.498218,15.692114,14.516528,11.709106,17.995654,13.184424
PI041-03-03,14.552490,15.759033,15.256470,15.368188,13.874780,11.200000,17.656470,12.684692
PI041-05-04,-2.363037,-1.811646,-2.338623,-1.990283,-2.335571,-4.787354,-2.153882,-4.023315
PI041-05-06,3.688623,4.649170,4.102356,4.091675,3.516089,0.810519,4.502356,2.179736
PI041-07-01,13.548462,14.454077,13.841650,14.263232,12.951514,9.992810,15.403174,12.418433
PI041-07-02,12.960449,14.075220,13.496582,13.884375,12.571130,9.383655,15.361975,11.285132


And here are two examples of the associated corrected melt compositions

In [19]:
corrected_inclusions["blundy2020"]

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,48.955334,14.022141,7.641438,9.712209,10.271092,3.604432,0.672949,0.138760,2.457926,0.279514,0.000000,0.606146,1.388979,0.076052,0.140278,0.032750,100.0
PI032-04-02,48.460278,14.603073,7.601481,9.370177,10.365700,3.443402,0.908394,0.140996,2.562524,0.324912,0.000000,0.649526,1.285696,0.081032,0.159781,0.043028,100.0
PI041-02-02,49.046604,16.834424,5.203108,9.125144,10.209750,3.762681,1.070413,0.157894,2.785146,0.557190,0.000000,0.460744,0.651006,0.047327,0.067554,0.021013,100.0
PI041-03-01,45.810872,15.412157,7.884103,10.766976,10.706980,3.273724,1.119966,0.117440,3.084757,0.539993,0.000000,0.780375,0.286965,0.077771,0.084975,0.052946,100.0
PI041-03-03,45.216710,15.599375,7.799072,10.966409,10.799270,3.350634,1.144690,0.094924,3.198585,0.507751,0.000000,0.805039,0.305039,0.080113,0.077938,0.054452,100.0
PI041-05-04,47.723380,18.466824,4.231369,9.340294,9.407433,4.569249,1.596153,0.143239,2.470414,0.819818,0.000000,0.506730,0.458631,0.087185,0.120646,0.058635,100.0
PI041-05-06,46.333280,16.835696,5.203644,8.856692,11.489733,3.969784,1.405488,0.173690,3.604942,0.614530,0.000000,0.634948,0.581054,0.106557,0.127094,0.062869,100.0
PI041-07-01,45.738312,15.075758,7.649539,9.588265,11.629978,3.098291,1.259229,0.150573,3.495990,0.549127,0.000000,0.462886,1.010255,0.074323,0.161731,0.055742,100.0
PI041-07-02,45.816049,15.378545,7.431276,9.954694,11.444496,3.149679,1.343152,0.139551,3.490226,0.608350,0.000000,0.367525,0.608517,0.072654,0.138638,0.056647,100.0


In [20]:
corrected_inclusions["saper2022"]

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,49.160985,14.315753,6.748100,9.907114,10.281719,3.679954,0.687049,0.133303,2.509426,0.285370,0.000000,0.618846,1.418081,0.077646,0.143217,0.033436,100.0
PI032-04-02,48.653312,14.934508,6.677632,9.573917,10.351842,3.521490,0.928994,0.136001,2.620636,0.332281,0.000000,0.664256,1.314852,0.082870,0.163404,0.044004,100.0
PI041-02-02,49.207466,17.085474,4.488479,9.254486,10.249549,3.818915,1.086410,0.150643,2.826770,0.565517,0.000000,0.467630,0.660736,0.048035,0.068564,0.021327,100.0
PI041-03-01,45.963429,15.780287,6.893711,11.015040,10.711034,3.352174,1.146804,0.112131,3.158679,0.552933,0.000000,0.799075,0.293842,0.079634,0.087011,0.054215,100.0
PI041-03-03,45.387804,16.009350,6.729923,11.243564,10.765915,3.439049,1.174896,0.087881,3.282988,0.521150,0.000000,0.826282,0.313089,0.082227,0.079995,0.055889,100.0
PI041-05-04,47.869489,18.816018,3.443179,9.508771,9.338272,4.655860,1.626408,0.134228,2.517241,0.835358,0.000000,0.516335,0.467325,0.088837,0.122933,0.059746,100.0
PI041-05-06,46.489925,17.215214,4.290470,9.048226,11.432423,4.059707,1.437325,0.164878,3.686600,0.628450,0.000000,0.649331,0.594216,0.108971,0.129972,0.064293,100.0
PI041-07-01,45.854257,15.505572,6.562708,9.849534,11.626393,3.186811,1.295205,0.143469,3.595872,0.564816,0.000000,0.476111,1.039119,0.076447,0.166352,0.057335,100.0
PI041-07-02,45.934165,15.816025,6.363440,10.226790,11.410105,3.239394,1.381410,0.133936,3.589641,0.625679,0.000000,0.377994,0.625850,0.074723,0.142587,0.058260,100.0
